In [1]:
#Загрузим необходимые библиотеки
from google.colab import drive
import numpy as np
from itertools import chain
drive.mount('/content/gdrive')
import pandas as pd
from collections import Counter

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
#Загрузим исходные данные с внешнего диска
vacancies_da = pd.read_excel('/content/gdrive/MyDrive/Colab Notebooks/vacancies_da.xlsx')

In [3]:
#приведем анализируемые поля к строчному формату
vacancies_da['name'] = vacancies_da['name'].str.lower()
vacancies_da['employer'] = vacancies_da['employer'].str.lower()
vacancies_da['key_skills'] = vacancies_da['key_skills'].str.lower()
vacancies_da['key_skills_from_key_skills_field'] = vacancies_da['key_skills_from_key_skills_field'].str.lower()
vacancies_da['hard_skills_from_description'] = vacancies_da['hard_skills_from_description'].str.lower()
vacancies_da['soft_skills_from_description'] = vacancies_da['soft_skills_from_description'].str.lower()

display(vacancies_da.head(5))

,id,name,published_at,alternate_url,type,employer,department,area,experience,key_skills,schedule,employment,description,description_lemmatized,salary_from,salary_to,salary_bin,key_skills_from_key_skills_field,hard_skills_from_description,soft_skills_from_description
0,77320677,управляющий эксперт по аналитике данных,2024-03-18 19:39:33,https://hh.ru/vacancy/77320677,Открытая,псб (пао «промсвязьбанк»),"ПАО «Промсвязьбанк», ИТ",Москва,Middle (3-6 years),"['бизнес-анализ', 'анализ бизнес показателей',...",Полный день,Полная занятость,на масштабный проект по созданию единого корпо...,масштабный проект создание единый корпоративны...,NaN,NaN,ЗП не указана,"финансовый анализ, бизнес-анализ, финансовая о...",NaN,документация
1,77963716,системный аналитик (взаимодействие между абоне...,2024-03-17 14:20:13,https://hh.ru/vacancy/77963716,Открытая,яковлев,NaN,Москва,Junior+ (1-3 years),"['apache, php, mysql', 'ethernet ieee 802.3', ...",Полный день,Полная занятость,системный аналитик (инф. взаимодействие между ...,системный аналитик инф взаимодействие абонент ...,NaN,NaN,ЗП не указана,"icd, afdx, excel, макросы vba, python, etherne...",NaN,NaN
2,79889558,аналитик данных в группу управления метаданным...,2024-03-16 14:48:58,https://hh.ru/vacancy/79889558,Открытая,мтс,"«МТС», Digital",Москва,Junior+ (1-3 years),"['data catalog', 'data governance', 'dmbok dam...",Полный день,Полная занятость,мтс – это мультисервисная цифровая экосистема....,мтс мультисервисный цифровой экосистема создав...,NaN,NaN,ЗП не указана,"data catalog, dmbok dama, business glossary, d...",NaN,NaN
3,82901165,data-аналитик,2024-03-25 10:33:57,https://hh.ru/vacancy/82901165,Открытая,элрос,NaN,Владимир,Junior+ (1-3 years),"['spark', 'data analysis', 'теория вероятносте...",Полный день,Полная занятость,наши преимущества: аккредитованная ит-компан...,наш преимущество аккредитовать ита компания си...,NaN,NaN,ЗП не указана,"python, математическая статистика, sql, spark,...",NaN,NaN
4,85598149,аналитик данных (bi),2024-03-27 09:08:54,https://hh.ru/vacancy/85598149,Открытая,evyap international russia,NaN,Москва,Junior+ (1-3 years),"['ms access', 'управление проектами', 'olap (o...",Полный день,Полная занятость,"evyap rus – международная fmcg компания, произ...",evyap rus международный fmcg компания производ...,NaN,NaN,ЗП не указана,"business intelligence systems, ms excel, ms ac...",NaN,документация


In [4]:
#Изучим перечень key_skills для обработки
vacancies_skills = vacancies_da
vacancies_skills['key_skills'] = vacancies_skills['key_skills'].apply(eval)
all_skills = []
for skills_list in vacancies_skills['key_skills']:
    all_skills.extend(skills_list)
    skills_df = pd.DataFrame({'Skills': all_skills})

print(set(all_skills))

{'dax', 'sql', 'библиотеки python', 'аналитика закупок', 'anaconda', 'ms project', 'xsd/json schema', 'superset', 'siddhi', 'oracle exadata', 'база данных: olap', 'pytorch', 'hive', 'bi-отчетность', 'составление отчетности', 'xml/json', 'jira', 'аналитические хранилища данных', 'кредитные риски', 'ms access', 'powerbi', 'erp systems', 'qgis', 'строительство', 'навыки презентации', 'ml-прогнозирование', 'google big query', 'e-commerce', 'gitlab', 'power bi', 'bokeh', 'анти фрод', 'grafana', 'маркетинговые исследования', 'проведение презентаций', 'грамотная речь и письмо', 'spring framework', 'ms outlook', 'аналитик data governance', 'статистика', 'google data studio', 'icd', 'исследования рынка', 'принятие решений', 'rest', 'afdx', 'spss', 'поиск информации в интернет', 'agile', 'безопасность данных', 'аудит', 'разработка фт', 'git', 'google таблицы', 'unit testing', 'выборка', 'маркетинговый анализ', 'проектная документация', 'kubernetes', 'pyspark', 'разработка технических заданий', '

In [5]:
vacancies_da['key_skills'] = vacancies_da['key_skills'].astype(str)

#Обработаем неявные дубликаты в поле key_skills
def replace_values(skills_count, replacements):
  for old_value, new_value in replacements:
      vacancies_da['key_skills'] = vacancies_da['key_skills'].str.replace(old_value, new_value)
  return skills_count

replacements = [
    ('a/b тестирование', 'тестирование'),
    ('a/b тесты', 'тестирование'),
    ('1с: торговля', '1c'),
    ('1с:мдм', '1c'),
    ('agile project management', 'agile'),
    ('apache hadoop', 'apache'),
    ('apache spark', 'apache'),
    ('bigdata', 'big data'),
    ('power bi', 'bi'),
    ('bi-отчетность', 'bi'),
    ('bi-отчет', 'bi'),
    ('powerbi', 'bi'),
    ('ms power bi', 'bi'),
    ('business intelligence systems', 'business intelligence'),
    ('data analyst', 'data analysis'),
    ('erp systems', 'erp'),
    ('ms excel', 'excel'),
    ('google таблицы', 'google docs'),
    ('java se', 'java'),
    ('javascript', 'java'),
    ('oracle db', 'oracle'),
    ('oracle exadata', 'oracle'),
    ('ms sql', 'sql'),
    ('oracle pl/sql', 'sql'),
    ('pl/sql', 'sql'),
    ('postgesql', 'sql'),
    ('postgresql', 'sql'),
    ('mysql', 'sql'),
    ('power query m', 'power query'),
    ('qlik sense', 'qlicksense'),
    ('аналитик данных', 'аналитика'),
    ('анализ данных', 'аналитика'),
    ('аналитический склад ума', 'аналитические способности'),
    ('аналитическое мышление', 'аналитические способности'),
    ('оптимизация бизнес процессов', 'оптимизация бизнес-процессов'),
    ('работа с базами данных', 'базы данных')
]

vacancies_da = replace_values(vacancies_da, replacements)
display(vacancies_da .head(10))





,id,name,published_at,alternate_url,type,employer,department,area,experience,key_skills,schedule,employment,description,description_lemmatized,salary_from,salary_to,salary_bin,key_skills_from_key_skills_field,hard_skills_from_description,soft_skills_from_description
0,77320677,управляющий эксперт по аналитике данных,2024-03-18 19:39:33,https://hh.ru/vacancy/77320677,Открытая,псб (пао «промсвязьбанк»),"ПАО «Промсвязьбанк», ИТ",Москва,Middle (3-6 years),"['бизнес-анализ', 'анализ бизнес показателей',...",Полный день,Полная занятость,на масштабный проект по созданию единого корпо...,масштабный проект создание единый корпоративны...,NaN,NaN,ЗП не указана,"финансовый анализ, бизнес-анализ, финансовая о...",NaN,документация
1,77963716,системный аналитик (взаимодействие между абоне...,2024-03-17 14:20:13,https://hh.ru/vacancy/77963716,Открытая,яковлев,NaN,Москва,Junior+ (1-3 years),"['apache, php, sql', 'ethernet ieee 802.3', 'a...",Полный день,Полная занятость,системный аналитик (инф. взаимодействие между ...,системный аналитик инф взаимодействие абонент ...,NaN,NaN,ЗП не указана,"icd, afdx, excel, макросы vba, python, etherne...",NaN,NaN
2,79889558,аналитик данных в группу управления метаданным...,2024-03-16 14:48:58,https://hh.ru/vacancy/79889558,Открытая,мтс,"«МТС», Digital",Москва,Junior+ (1-3 years),"['data catalog', 'data governance', 'dmbok dam...",Полный день,Полная занятость,мтс – это мультисервисная цифровая экосистема....,мтс мультисервисный цифровой экосистема создав...,NaN,NaN,ЗП не указана,"data catalog, dmbok dama, business glossary, d...",NaN,NaN
3,82901165,data-аналитик,2024-03-25 10:33:57,https://hh.ru/vacancy/82901165,Открытая,элрос,NaN,Владимир,Junior+ (1-3 years),"['spark', 'data analysis', 'теория вероятносте...",Полный день,Полная занятость,наши преимущества: аккредитованная ит-компан...,наш преимущество аккредитовать ита компания си...,NaN,NaN,ЗП не указана,"python, математическая статистика, sql, spark,...",NaN,NaN
4,85598149,аналитик данных (bi),2024-03-27 09:08:54,https://hh.ru/vacancy/85598149,Открытая,evyap international russia,NaN,Москва,Junior+ (1-3 years),"['ms access', 'управление проектами', 'olap (o...",Полный день,Полная занятость,"evyap rus – международная fmcg компания, произ...",evyap rus международный fmcg компания производ...,NaN,NaN,ЗП не указана,"business intelligence systems, ms excel, ms ac...",NaN,документация
5,86212079,аналитик данных bi отдела автоматизации (гибри...,2024-03-29 16:50:53,https://hh.ru/vacancy/86212079,Открытая,группа компаний агроэко,NaN,Воронеж,Junior+ (1-3 years),['документация'],Полный день,Полная занятость,вакансия открыта в связи с расширением отдела ...,вакансия открывать связь расширение отдел авто...,NaN,NaN,ЗП не указана,NaN,NaN,документация
6,87643118,junior data engineer/ bi аналитик (дашборды),2024-03-20 14:11:06,https://hh.ru/vacancy/87643118,Открытая,гбуз «научно-практический клинический центр ди...,NaN,Москва,Junior+ (1-3 years),"['аналитические способности', 'английский язык...",Полный день,Полная занятость,обязанности построение и оптимизация etl скри...,обязанность построение оптимизация etl скрипт ...,100000.0,NaN,ЗП не указана,"английский язык, научные исследования, базы да...",NaN,NaN
7,87684971,аналитик данных,2024-03-18 08:23:07,https://hh.ru/vacancy/87684971,Открытая,кадровый центр департамента здравоохранения го...,NaN,Москва,Junior+ (1-3 years),"['аналитические способности', 'excel', 'ms pow...",Полный день,Полная занятость,обязанности: анализ данных по заданным критер...,обязанность анализ данные заданный критерий по...,100000.0,NaN,ЗП не указана,"ms excel, аналитика, ms powerpoint, анализ дан...",NaN,NaN
8,88605313,data analyst,2024-03-12 21:40:27,https://hh.ru/vacancy/88605313,Открытая,бланк – банк для бизнеса,NaN,Москва,Junior+ (1-3 years),"['bi', 'bi', 'tableau', 'sql', 'sql server', '...",Удаленная работа,Полная занятость,"привет!мы – бланк, современный life-first моби...",привет бланк современный life first мобильный ...,NaN,NaN,ЗП не у

In [6]:
#Превратим данные в поле в список списков  сбор и анализ информации
vacancies_da['key_skills'] = vacancies_da['key_skills'].apply(eval)

In [7]:
#Получим срез key_skills для всех уровней. Для этого напием функцию сортировки датафрейма по уровню соискателя

def count_unique_skills_by_experience(experience):
    vacancies_da_filtered = vacancies_da[vacancies_da['experience'] == experience]
    all_skills = []
    for skills_list in vacancies_da_filtered['key_skills']:
        all_skills.extend(skills_list)
    skills_df = pd.DataFrame({'Skills': all_skills})
    skills_count = skills_df['Skills'].value_counts().reset_index(name='Count')
    return skills_count

In [8]:
#Посмотрим на топ 15 скилов уровня Junior
skills_count_j = count_unique_skills_by_experience("Junior (no experince)")
print('Топ 15 скилов уровня Junior:',
    '\n', skills_count_j.head(15))

Топ 15 скилов уровня Junior: 
                                  Skills  Count
0                                   sql     13
1                          документация     12
2                                 excel      8
3                                python      7
4             аналитические способности      6
5                             аналитика      5
6                          коммуникация      5
7   работа с большим объемом информации      4
8                           базы данных      4
9                         ms powerpoint      4
10                               pandas      4
11                      английский язык      2
12                     работа в команде      2
13           аналитические исследования      2
14                                   bi      2


In [9]:
print('Топ 5 Hard-skils по степени важности для работодателя на уровне Junior:'
      '\n 1.sql'
      '\n 2. excel'
      '\n 3. python'
      '\n 4. аналитика'
      '\n 5. ms powerpoint')
print()
print('Топ 5 Soft-skils:'
      '\n 1. документация'
      '\n 2. аналитические способности'
      '\n 3. коммуникация'
      '\n 4. работа с большим объемом информации'
      '\n 5. базы данных')

Топ 5 Hard-skils по степени важности для работодателя на уровне Junior:
 1.sql
 2. excel
 3. python
 4. аналитика
 5. ms powerpoint

Топ 5 Soft-skils:
 1. документация
 2. аналитические способности
 3. коммуникация
 4. работа с большим объемом информации
 5. базы данных


In [10]:
# Выведем аналогичный свод скилов для остальных уровней
#топ 15 скилов уровня Junior+
skills_count_j1 = count_unique_skills_by_experience("Junior+ (1-3 years)")
#print('Топ 15 скилов уровня Junior+:',
    #'\n', skills_count_j1.head(15))
#print()
print('Топ 5 Hard-skils по степени важности для работодателя на уровне Junior+:'
      '\n 1. sql'
      '\n 2. аналитика'
      '\n 3. python'
      '\n 4. bi'
      '\n 5. excel')
print()
print('Топ 5 Soft-skils:'
      '\n 1. аналитические способности'
      '\n 2. документация'
      '\n 3. работа с большим объемом информации'
      '\n 4. коммуникация'
      '\n 5. бизнес-анализ')

Топ 5 Hard-skils по степени важности для работодателя на уровне Junior+:
 1. sql
 2. аналитика
 3. python
 4. bi
 5. excel

Топ 5 Soft-skils:
 1. аналитические способности
 2. документация
 3. работа с большим объемом информации
 4. коммуникация
 5. бизнес-анализ


In [11]:
#топ 15 скилов уровня Middle
skills_count_m = count_unique_skills_by_experience("Middle (3-6 years)")
#print('Топ 15 скилов уровня Middle:',
    #'\n', skills_count_m.head(15))

#print()
print('Топ 5 Hard-skils по степени важности для работодателя на уровне Middle:'
      '\n 1. sql'
      '\n 2. python'
      '\n 3. pandas'
      '\n 4. аналитика'
      '\n 5. bi')
print()
print('Топ 5 Soft-skils:'
      '\n 1. документация'
      '\n 2. базы данных'
      '\n 3. аналитические способности'
      '\n 4. коммуникация'
      '\n 5. математическая статистика')

Топ 5 Hard-skils по степени важности для работодателя на уровне Middle:
 1. sql
 2. python
 3. pandas
 4. аналитика
 5. bi

Топ 5 Soft-skils:
 1. документация
 2. базы данных
 3. аналитические способности
 4. коммуникация
 5. математическая статистика


In [12]:
#топ 15 скилов уровня Senior
skills_count_s = count_unique_skills_by_experience("Senior (6+ years)")
print('Топ 15 скилов уровня Senior:',
    '\n', skills_count_s.head(15))

#print()
print('В данном грейде всго одна вакансия. Выборка недостаточна для определения важнейших скилов')


Топ 15 скилов уровня Senior: 
           Skills  Count
0          excel      1
1  бизнес-анализ      1
2   data science      1
3       big data      1
4      data lake      1
5  проактивность      1
6   коммуникация      1
В данном грейде всго одна вакансия. Выборка недостаточна для определения важнейших скилов


In [13]:
#Посмотрим какую долю вакансий предлагают разные компании по грейдам:
count_by_experience = vacancies_da['experience'].value_counts().reset_index(name='Count')
count_by_experience['percent'] = (count_by_experience['Count'] / count_by_experience['Count'].sum()).apply('{:,.2%}'.format)
display('Распределение вакансий по требуемому уровню:',
    '\n', count_by_experience.sort_values(by='experience', ascending=True).reset_index(drop=True))

'Распределение вакансий по требуемому уровню:'

'\n'

,experience,Count,percent
0,Junior (no experince),36,8.63%
1,Junior+ (1-3 years),243,58.27%
2,Middle (3-6 years),137,32.85%
3,Senior (6+ years),1,0.24%


In [14]:
# Определеним типичное место работы для Бизнес-аналитика по следующим параметрам: ТОП-работодателей, зарплата, тип занятости, график работы.
company_by_experience = vacancies_da.groupby(['experience', 'employer'])['employer'].count().rename('count_of_companies').to_frame().sort_values(['experience', 'count_of_companies'], ascending=[True, False])
print('Топ компаний в поиске по грейдам Data-аналитика:')
display(company_by_experience.groupby(level=0).head(5))

Топ компаний в поиске по грейдам Data-аналитика:


count_of_companies
experience            employer                                       
Junior (no experince) axenix (ранее accenture)                      6
                      сбер                                          5
                      правительство москвы                          2
                      changellenge                                  1
                      cityads media                                 1
Junior+ (1-3 years)   сбер                                         45
                      мтс                                           7
                      ozon                                          5
                      центр финансовых технологий                   5
                      банк втб (пао)                                4
Middle (3-6 years)    сбер                                         15
                      т1                                            7
                      wildberries                                   6
                      inguru.ru                                     5
                      мтс                                           5
Senior (6+ years)     jsa group                                     1

In [15]:
salary_by_experience = round(vacancies_da.groupby(['experience'])[['salary_from', 'salary_to']].mean())
display('Средняя зарплата по грейдам Data-аналитика:',
    salary_by_experience)

'Средняя зарплата по грейдам Data-аналитика:'

,salary_from,salary_to
experience,,
Junior (no experince),62554.0,87011.0
Junior+ (1-3 years),92465.0,151717.0
Middle (3-6 years),169500.0,299108.0
Senior (6+ years),NaN,NaN


In [16]:
salary_by_experience_null = vacancies_da.groupby(['experience', 'salary_bin'])['salary_bin'].count().rename('count_of_salary_bin').to_frame().sort_values(['experience', 'count_of_salary_bin'], ascending=[True, False])
salary_by_experience_null['salary_bin_percent'] = (salary_by_experience_null['count_of_salary_bin'] / salary_by_experience_null.groupby('experience')['count_of_salary_bin'].sum()).apply('{:,.2%}'.format)
display('Доля вакансий с зарплатной вилкой:',
    salary_by_experience_null)

'Доля вакансий с зарплатной вилкой:'

count_of_salary_bin  \
experience            salary_bin                                       
Junior (no experince) ЗП не указана                               32   
                      Меньше 100 тысяч                             3   
                      От 100 тысяч до 200 тысяч                    1   
Junior+ (1-3 years)   ЗП не указана                              230   
                      От 100 тысяч до 200 тысяч                    6   
                      Меньше 100 тысяч                             3   
                      От 200 тысяч до 300 тысяч                    3   
                      Больше 300 тысяч                             1   
Middle (3-6 years)    ЗП не указана                              124   
                      Больше 300 тысяч                            10   
                      От 200 тысяч до 300 тысяч                    2   
                      От 100 тысяч до 200 тысяч                    1   
Senior (6+ years)     ЗП не указана                                1   

                                                salary_bin_percent  
experience            salary_bin                                    
Junior (no experince) ЗП не указана                         88.89%  
                      Меньше 100 тысяч                       8.33%  
                      От 100 тысяч до 200 тысяч              2.78%  
Junior+ (1-3 years)   ЗП не указана                         94.65%  
                      От 100 тысяч до 200 тысяч              2.47%  
                      Меньше 100 тысяч                       1.23%  
                      От 200 тысяч до 300 тысяч              1.23%  
                      Больше 300 тысяч                       0.41%  
Middle (3-6 years)    ЗП не указана                         90.51%  
                      Больше 300 тысяч                       7.30%  
                      От 200 тысяч до 300 тысяч              1.46%  
                      От 100 тысяч до 200 тысяч              0.73%  
Senior (6+ years)     ЗП не указана                        100.00%

In [17]:
time_by_experience = vacancies_da.groupby(['experience', 'employment'])['employment'].count().rename('count_of_employment').to_frame().sort_values(['experience', 'count_of_employment'], ascending=[True, False])
print('Тип занятости по грейдам Data-аналитика:')
time_by_experience['employment_percent'] = (time_by_experience['count_of_employment'] / time_by_experience.groupby('experience')['count_of_employment'].sum()).apply('{:,.2%}'.format)

display(time_by_experience.groupby(level=0).head(5))

Тип занятости по грейдам Data-аналитика:


count_of_employment  \
experience            employment                                 
Junior (no experince) Полная занятость                      31   
                      Стажировка                             4   
                      Частичная занятость                    1   
Junior+ (1-3 years)   Полная занятость                     239   
                      Частичная занятость                    3   
                      Проектная работа                       1   
Middle (3-6 years)    Полная занятость                     137   
Senior (6+ years)     Полная занятость                       1   

                                          employment_percent  
experience            employment                              
Junior (no experince) Полная занятость                86.11%  
                      Стажировка                      11.11%  
                      Частичная занятость              2.78%  
Junior+ (1-3 years)   Полная занятость                98.35%  
                      Частичная занятость              1.23%  
                      Проектная работа                 0.41%  
Middle (3-6 years)    Полная занятость               100.00%  
Senior (6+ years)     Полная занятость               100.00%

In [18]:
time_by_experience = vacancies_da.groupby(['experience', 'schedule'])['schedule'].count().rename('count_of_schedule').to_frame().sort_values(['experience', 'count_of_schedule'], ascending=[True, False])
print('График работы по грейдам Data-аналитика:')
time_by_experience['schedule_percent'] = (time_by_experience['count_of_schedule'] / time_by_experience.groupby('experience')['count_of_schedule'].sum()).apply('{:,.2%}'.format)
display(time_by_experience.groupby(level=0).head(5))

График работы по грейдам Data-аналитика:


count_of_schedule schedule_percent
experience            schedule                                            
Junior (no experince) Полный день                      33           91.67%
                      Удаленная работа                  2            5.56%
                      Сменный график                    1            2.78%
Junior+ (1-3 years)   Полный день                     205           84.36%
                      Удаленная работа                 33           13.58%
                      Гибкий график                     4            1.65%
                      Сменный график                    1            0.41%
Middle (3-6 years)    Полный день                     105           76.64%
                      Удаленная работа                 29           21.17%
                      Гибкий график                     3            2.19%
Senior (6+ years)     Полный день                       1          100.00%